Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [28]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [29]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## Logistic Regression

In [10]:
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases  = tf.Variable(tf.zeros([num_labels]))
  
  # Regularization loss
  beta    = tf.Variable(tf.zeros([1]))  
  l2_loss = 5e-4 * tf.nn.l2_loss(weights) 

  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + l2_loss)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [11]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.930811
Training accuracy: 10.4%
Validation accuracy: 12.4%
Loss at step 100: 3.623012
Training accuracy: 71.8%
Validation accuracy: 70.8%
Loss at step 200: 3.038037
Training accuracy: 75.2%
Validation accuracy: 73.8%
Loss at step 300: 2.674638
Training accuracy: 76.6%
Validation accuracy: 75.1%
Loss at step 400: 2.404722
Training accuracy: 77.5%
Validation accuracy: 75.7%
Loss at step 500: 2.189553
Training accuracy: 78.1%
Validation accuracy: 76.2%
Loss at step 600: 2.011445
Training accuracy: 78.8%
Validation accuracy: 76.6%
Loss at step 700: 1.860391
Training accuracy: 79.5%
Validation accuracy: 76.8%
Loss at step 800: 1.730084
Training accuracy: 80.2%
Validation accuracy: 77.1%
Test accuracy: 84.4%


## 1-hidden layer Neural Network

In [12]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

batch_size = 128
num_relus  = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset  = tf.constant(test_dataset)
  
  # Variables.
  weights1 = weight_variable([image_size * image_size, num_relus])
  biases1  = bias_variable([num_relus])
  weights2 = weight_variable([num_relus, num_labels])
  biases2  = bias_variable([num_labels])
    
  # Regularization loss
  beta    = tf.Variable(tf.zeros([1]))  
  l2_loss = 5e-4 * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))   

  
  # Training computation.
  to_hidden   = tf.matmul(tf_train_dataset, weights1) + biases1
  from_hidden = tf.nn.relu(to_hidden)
  logits      = tf.matmul(from_hidden, weights2) + biases2
  loss        = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + l2_loss)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
                     tf.matmul(
                     tf.nn.relu(
                     tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
                    tf.matmul(
                    tf.nn.relu(
                    tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [13]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.504155
Minibatch accuracy: 6.2%
Validation accuracy: 27.3%
Minibatch loss at step 500: 2.111861
Minibatch accuracy: 77.3%
Validation accuracy: 85.6%
Minibatch loss at step 1000: 1.413004
Minibatch accuracy: 86.7%
Validation accuracy: 85.5%
Minibatch loss at step 1500: 1.062768
Minibatch accuracy: 91.4%
Validation accuracy: 87.8%
Minibatch loss at step 2000: 1.119647
Minibatch accuracy: 85.2%
Validation accuracy: 87.9%
Minibatch loss at step 2500: 0.852096
Minibatch accuracy: 90.6%
Validation accuracy: 88.8%
Minibatch loss at step 3000: 0.860167
Minibatch accuracy: 88.3%
Validation accuracy: 88.6%
Test accuracy: 94.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

## Reduce training set from 200K to 2K

In [23]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

batch_size = 128
num_relus  = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset  = tf.constant(test_dataset)
  
  # Variables.
  weights1 = weight_variable([image_size * image_size, num_relus])
  biases1  = bias_variable([num_relus])
  weights2 = weight_variable([num_relus, num_labels])
  biases2  = bias_variable([num_labels])
    
  # Regularization loss
  beta    = tf.Variable(tf.zeros([1]))  
  l2_loss = 5e-4 * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))   

  
  # Training computation.
  to_hidden   = tf.matmul(tf_train_dataset, weights1) + biases1
  from_hidden = tf.nn.relu(to_hidden)
  logits      = tf.matmul(from_hidden, weights2) + biases2
  loss        = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + l2_loss)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
                     tf.matmul(
                     tf.nn.relu(
                     tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
                    tf.matmul(
                    tf.nn.relu(
                    tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [21]:
# Reduce training set from 200K to 2K
train_dataset = train_dataset[:2000]
train_labels = train_labels[:2000]
print(train_dataset.shape,train_labels.shape)

(2000, 784) (2000, 10)


In [24]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6.305584
Minibatch accuracy: 7.8%
Validation accuracy: 32.0%
Minibatch loss at step 500: 1.256534
Minibatch accuracy: 100.0%
Validation accuracy: 81.3%
Minibatch loss at step 1000: 0.979508
Minibatch accuracy: 100.0%
Validation accuracy: 81.6%
Minibatch loss at step 1500: 0.776145
Minibatch accuracy: 100.0%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 0.611123
Minibatch accuracy: 100.0%
Validation accuracy: 81.8%
Minibatch loss at step 2500: 0.485294
Minibatch accuracy: 100.0%
Validation accuracy: 82.0%
Minibatch loss at step 3000: 0.390732
Minibatch accuracy: 100.0%
Validation accuracy: 81.9%
Test accuracy: 88.6%


## Increase batch size from 128 to 2000

In [17]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

batch_size = 2000
num_relus  = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset  = tf.constant(test_dataset)
  
  # Variables.
  weights1 = weight_variable([image_size * image_size, num_relus])
  biases1  = bias_variable([num_relus])
  weights2 = weight_variable([num_relus, num_labels])
  biases2  = bias_variable([num_labels])
    
  # Regularization loss
  beta    = tf.Variable(tf.zeros([1]))  
  l2_loss = 5e-4 * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))   

  
  # Training computation.
  to_hidden   = tf.matmul(tf_train_dataset, weights1) + biases1
  from_hidden = tf.nn.relu(to_hidden)
  logits      = tf.matmul(from_hidden, weights2) + biases2
  loss        = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + l2_loss)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
                     tf.matmul(
                     tf.nn.relu(
                     tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
                    tf.matmul(
                    tf.nn.relu(
                    tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [18]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6.114238
Minibatch accuracy: 9.6%
Validation accuracy: 44.7%
Minibatch loss at step 500: 1.586385
Minibatch accuracy: 89.8%
Validation accuracy: 87.6%
Minibatch loss at step 1000: 1.304788
Minibatch accuracy: 90.0%
Validation accuracy: 88.8%
Minibatch loss at step 1500: 1.102540
Minibatch accuracy: 90.3%
Validation accuracy: 89.5%
Minibatch loss at step 2000: 0.898823
Minibatch accuracy: 91.6%
Validation accuracy: 89.9%
Minibatch loss at step 2500: 0.754545
Minibatch accuracy: 93.8%
Validation accuracy: 90.2%
Minibatch loss at step 3000: 0.652367
Minibatch accuracy: 93.3%
Validation accuracy: 90.5%
Test accuracy: 95.2%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

## Reduced Dataset With Dropout

In [25]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

batch_size = 128
num_relus  = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset  = tf.constant(test_dataset)
  
  # Variables.
  weights1 = weight_variable([image_size * image_size, num_relus])
  biases1  = bias_variable([num_relus])
  weights2 = weight_variable([num_relus, num_labels])
  biases2  = bias_variable([num_labels])
    
  # Regularization loss
  beta    = tf.Variable(tf.zeros([1]))  
  l2_loss = 5e-4 * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))   

  
  # Training computation.
  to_hidden   = tf.matmul(tf_train_dataset, weights1) + biases1
  from_hidden = tf.nn.relu(to_hidden)

  # Introduce dropout before readout layer
  keep_prob = tf.placeholder(tf.float32)
  from_drop = tf.nn.dropout(from_hidden, keep_prob)
                           
  logits      = tf.matmul(from_drop, weights2) + biases2
  loss        = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + l2_loss)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
                     tf.matmul(
                     tf.nn.relu(
                     tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
                    tf.matmul(
                    tf.nn.relu(
                    tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [27]:
# Increase num_steps to 10001 from 3001
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 8.029771
Minibatch accuracy: 9.4%
Validation accuracy: 37.1%
Minibatch loss at step 500: 1.309707
Minibatch accuracy: 98.4%
Validation accuracy: 81.9%
Minibatch loss at step 1000: 1.001808
Minibatch accuracy: 100.0%
Validation accuracy: 82.3%
Minibatch loss at step 1500: 0.788273
Minibatch accuracy: 100.0%
Validation accuracy: 82.2%
Minibatch loss at step 2000: 0.625677
Minibatch accuracy: 100.0%
Validation accuracy: 82.5%
Minibatch loss at step 2500: 0.493522
Minibatch accuracy: 100.0%
Validation accuracy: 82.5%
Minibatch loss at step 3000: 0.396547
Minibatch accuracy: 100.0%
Validation accuracy: 82.4%
Minibatch loss at step 3500: 0.318557
Minibatch accuracy: 100.0%
Validation accuracy: 82.5%
Minibatch loss at step 4000: 0.260899
Minibatch accuracy: 100.0%
Validation accuracy: 82.4%
Minibatch loss at step 4500: 0.216015
Minibatch accuracy: 100.0%
Validation accuracy: 82.5%
Minibatch loss at step 5000: 0.180304
Minibatch accuracy: 100.0%
Validation

## Original Size Dataset with Dropout

In [30]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

batch_size = 128
num_relus  = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset  = tf.constant(test_dataset)
  
  # Variables.
  weights1 = weight_variable([image_size * image_size, num_relus])
  biases1  = bias_variable([num_relus])
  weights2 = weight_variable([num_relus, num_labels])
  biases2  = bias_variable([num_labels])
    
  # Regularization loss
  beta    = tf.Variable(tf.zeros([1]))  
  l2_loss = 5e-4 * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))   

  
  # Training computation.
  to_hidden   = tf.matmul(tf_train_dataset, weights1) + biases1
  from_hidden = tf.nn.relu(to_hidden)

  # Introduce dropout before readout layer
  keep_prob = tf.placeholder(tf.float32)
  from_drop = tf.nn.dropout(from_hidden, keep_prob)
                           
  logits      = tf.matmul(from_drop, weights2) + biases2
  loss        = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + l2_loss)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
                     tf.matmul(
                     tf.nn.relu(
                     tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
                    tf.matmul(
                    tf.nn.relu(
                    tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [31]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 7.408321
Minibatch accuracy: 13.3%
Validation accuracy: 22.5%
Minibatch loss at step 500: 2.278996
Minibatch accuracy: 75.8%
Validation accuracy: 84.3%
Minibatch loss at step 1000: 1.516135
Minibatch accuracy: 85.2%
Validation accuracy: 85.7%
Minibatch loss at step 1500: 1.079779
Minibatch accuracy: 91.4%
Validation accuracy: 86.6%
Minibatch loss at step 2000: 1.224036
Minibatch accuracy: 82.0%
Validation accuracy: 86.6%
Minibatch loss at step 2500: 0.905920
Minibatch accuracy: 87.5%
Validation accuracy: 87.7%
Minibatch loss at step 3000: 0.970840
Minibatch accuracy: 85.9%
Validation accuracy: 87.8%
Test accuracy: 93.6%


In [32]:
# Increase num_steps from 3001 to 10001
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 7.062184
Minibatch accuracy: 16.4%
Validation accuracy: 28.5%
Minibatch loss at step 500: 2.393376
Minibatch accuracy: 71.1%
Validation accuracy: 85.0%
Minibatch loss at step 1000: 1.542832
Minibatch accuracy: 85.2%
Validation accuracy: 86.3%
Minibatch loss at step 1500: 1.179118
Minibatch accuracy: 84.4%
Validation accuracy: 86.9%
Minibatch loss at step 2000: 1.239508
Minibatch accuracy: 82.0%
Validation accuracy: 86.9%
Minibatch loss at step 2500: 0.989349
Minibatch accuracy: 84.4%
Validation accuracy: 87.7%
Minibatch loss at step 3000: 1.031272
Minibatch accuracy: 84.4%
Validation accuracy: 87.9%
Minibatch loss at step 3500: 0.869526
Minibatch accuracy: 82.8%
Validation accuracy: 86.8%
Minibatch loss at step 4000: 0.561085
Minibatch accuracy: 89.8%
Validation accuracy: 87.6%
Minibatch loss at step 4500: 0.699754
Minibatch accuracy: 87.5%
Validation accuracy: 87.8%
Minibatch loss at step 5000: 0.695755
Minibatch accuracy: 83.6%
Validation accurac

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


## Add two more hidden layers

### Size the hidden layers - wide to narrow
### Add drop out to all hidden layers and input layers

In [50]:
# Use std = sqrt(2/n) where n is number of inputs in to the layer 
def weight_variable(shape,std):
  initial = tf.truncated_normal(shape, stddev=std)
  return tf.Variable(initial)

In [69]:
import math

batch_size     = 128
hlayer1_size   = 1024
hlayer2_size   = 512
hlayer3_size   = 256

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset  = tf.constant(test_dataset)
  
  # Variables.
  weights1 = weight_variable([image_size * image_size, hlayer1_size], math.sqrt(float(2.0/(image_size*image_size))))
  biases1  = bias_variable([hlayer1_size])
  weights2 = weight_variable([hlayer1_size, hlayer2_size], math.sqrt(float(2.0/(hlayer1_size))))
  biases2  = bias_variable([hlayer2_size])
  weights3 = weight_variable([hlayer2_size, hlayer3_size], math.sqrt(float(2.0/(hlayer2_size))))
  biases3  = bias_variable([hlayer3_size])
  weights4 = weight_variable([hlayer3_size, num_labels], math.sqrt(float(2.0/(hlayer3_size))))
  biases4  = bias_variable([num_labels])
    
  # Regularization loss
  beta    = tf.Variable(tf.zeros([1]))  
  l2_loss = 5e-4 * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3) + tf.nn.l2_loss(weights4))   

  
  # Training computation.
  # Introduce dropout for input layer
  keep_prob   = tf.placeholder(tf.float32)
  from_indrop = tf.nn.dropout(tf_train_dataset, keep_prob)
    
  to_hidden1   = tf.matmul(from_indrop, weights1) + biases1
  from_hidden1 = tf.nn.relu(to_hidden1)

  # Introduce dropout after hidden layers
  from_h1drop = tf.nn.dropout(from_hidden1, keep_prob)

  to_hidden2   = tf.matmul(from_h1drop, weights2) + biases2
  from_hidden2 = tf.nn.relu(to_hidden2)
  from_h2drop = tf.nn.dropout(from_hidden2, keep_prob)
    
  to_hidden3   = tf.matmul(from_h2drop, weights3) + biases3
  from_hidden3 = tf.nn.relu(to_hidden3)
  from_h3drop = tf.nn.dropout(from_hidden3, keep_prob)
                           
  logits      = tf.matmul(from_h3drop, weights4) + biases4
  loss        = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + l2_loss)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
                     tf.matmul(
                     tf.nn.relu(
                     tf.matmul(
                     tf.nn.relu(
                     tf.matmul(
                     tf.nn.relu(
                     tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3), weights4) + biases4)
  test_prediction = tf.nn.softmax(
                    tf.matmul(
                    tf.nn.relu(
                    tf.matmul(
                    tf.nn.relu(
                    tf.matmul(
                    tf.nn.relu(
                    tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3), weights4) + biases4)

In [68]:
num_steps = 40001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.8}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 5000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 3.187289
Minibatch accuracy: 13.3%
Validation accuracy: 11.1%
Minibatch loss at step 5000: 0.606372
Minibatch accuracy: 85.9%
Validation accuracy: 88.9%
Minibatch loss at step 10000: 0.481073
Minibatch accuracy: 86.7%
Validation accuracy: 89.9%
Minibatch loss at step 15000: 0.629993
Minibatch accuracy: 86.7%
Validation accuracy: 89.0%
Minibatch loss at step 20000: 0.381437
Minibatch accuracy: 93.0%
Validation accuracy: 89.6%
Minibatch loss at step 25000: 0.503030
Minibatch accuracy: 91.4%
Validation accuracy: 89.8%
Minibatch loss at step 30000: 0.527659
Minibatch accuracy: 88.3%
Validation accuracy: 89.3%
Minibatch loss at step 35000: 0.562465
Minibatch accuracy: 85.2%
Validation accuracy: 90.1%
Minibatch loss at step 40000: 0.454538
Minibatch accuracy: 91.4%
Validation accuracy: 89.8%
Test accuracy: 95.2%


In [70]:
num_steps = 40001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.8}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 5000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 3.231401
Minibatch accuracy: 9.4%
Validation accuracy: 15.1%
Minibatch loss at step 5000: 0.552377
Minibatch accuracy: 89.1%
Validation accuracy: 89.4%
Minibatch loss at step 10000: 0.515443
Minibatch accuracy: 85.9%
Validation accuracy: 89.8%
Minibatch loss at step 15000: 0.651468
Minibatch accuracy: 90.6%
Validation accuracy: 89.3%
Minibatch loss at step 20000: 0.355680
Minibatch accuracy: 95.3%
Validation accuracy: 89.8%
Minibatch loss at step 25000: 0.543242
Minibatch accuracy: 88.3%
Validation accuracy: 90.0%
Minibatch loss at step 30000: 0.509155
Minibatch accuracy: 88.3%
Validation accuracy: 90.0%
Minibatch loss at step 35000: 0.508750
Minibatch accuracy: 89.1%
Validation accuracy: 89.9%
Minibatch loss at step 40000: 0.462831
Minibatch accuracy: 92.2%
Validation accuracy: 90.0%
Test accuracy: 95.0%


## Introduce Learning Rate Decay

In [71]:
import math

batch_size     = 128
hlayer1_size   = 1024
hlayer2_size   = 512
hlayer3_size   = 256

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset  = tf.constant(test_dataset)
  
  # Variables.
  weights1 = weight_variable([image_size * image_size, hlayer1_size], math.sqrt(float(2.0/(image_size*image_size))))
  biases1  = bias_variable([hlayer1_size])
  weights2 = weight_variable([hlayer1_size, hlayer2_size], math.sqrt(float(2.0/(hlayer1_size))))
  biases2  = bias_variable([hlayer2_size])
  weights3 = weight_variable([hlayer2_size, hlayer3_size], math.sqrt(float(2.0/(hlayer2_size))))
  biases3  = bias_variable([hlayer3_size])
  weights4 = weight_variable([hlayer3_size, num_labels], math.sqrt(float(2.0/(hlayer3_size))))
  biases4  = bias_variable([num_labels])
    
  # Regularization loss
  beta    = tf.Variable(tf.zeros([1]))  
  l2_loss = 5e-4 * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3) + tf.nn.l2_loss(weights4))   

  
  # Training computation.
  # Introduce dropout for input layer
  keep_prob   = tf.placeholder(tf.float32)
  from_indrop = tf.nn.dropout(tf_train_dataset, keep_prob)
    
  to_hidden1   = tf.matmul(from_indrop, weights1) + biases1
  from_hidden1 = tf.nn.relu(to_hidden1)

  # Introduce dropout after hidden layers
  from_h1drop = tf.nn.dropout(from_hidden1, keep_prob)

  to_hidden2   = tf.matmul(from_h1drop, weights2) + biases2
  from_hidden2 = tf.nn.relu(to_hidden2)
  from_h2drop = tf.nn.dropout(from_hidden2, keep_prob)
    
  to_hidden3   = tf.matmul(from_h2drop, weights3) + biases3
  from_hidden3 = tf.nn.relu(to_hidden3)
  from_h3drop = tf.nn.dropout(from_hidden3, keep_prob)
                           
  logits      = tf.matmul(from_h3drop, weights4) + biases4
  loss        = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + l2_loss)
  
  # Decaying Learning Rate
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.3, global_step, 3500, 0.86, staircase=True)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
                     tf.matmul(
                     tf.nn.relu(
                     tf.matmul(
                     tf.nn.relu(
                     tf.matmul(
                     tf.nn.relu(
                     tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3), weights4) + biases4)
  test_prediction = tf.nn.softmax(
                    tf.matmul(
                    tf.nn.relu(
                    tf.matmul(
                    tf.nn.relu(
                    tf.matmul(
                    tf.nn.relu(
                    tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3), weights4) + biases4)

In [72]:
num_steps = 40001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.8}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 5000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 3.385790
Minibatch accuracy: 4.7%
Validation accuracy: 28.5%
Minibatch loss at step 5000: 0.638171
Minibatch accuracy: 86.7%
Validation accuracy: 89.3%
Minibatch loss at step 10000: 0.459493
Minibatch accuracy: 89.8%
Validation accuracy: 90.2%
Minibatch loss at step 15000: 0.529144
Minibatch accuracy: 89.1%
Validation accuracy: 90.6%
Minibatch loss at step 20000: 0.329098
Minibatch accuracy: 96.1%
Validation accuracy: 90.8%
Minibatch loss at step 25000: 0.400082
Minibatch accuracy: 92.2%
Validation accuracy: 91.4%
Minibatch loss at step 30000: 0.398600
Minibatch accuracy: 92.2%
Validation accuracy: 91.3%
Minibatch loss at step 35000: 0.360615
Minibatch accuracy: 93.0%
Validation accuracy: 91.5%
Minibatch loss at step 40000: 0.288343
Minibatch accuracy: 94.5%
Validation accuracy: 91.7%
Test accuracy: 96.5%
